In [1]:
from loading_data import *

/Users/maximeswagel/Library/Mobile Documents/com~apple~CloudDocs/Documents/EPFL/DTU/Courses/SecondSemester/MachineLearning/Assignment2/scripts

Location of the south_african_heart_disease.xls file: /Users/maximeswagel/Library/Mobile Documents/com~apple~CloudDocs/Documents/EPFL/DTU/Courses/SecondSemester/MachineLearning/Assignment2/data/south_african_heart_disease.xls


In [2]:
#First we want to normalize and transform our data.

from scipy.stats import zscore

normalised_X = np.copy(X)
#transform
normalised_X[:,6] = np.log(1 + X[:,6]) #add 1 because some alcohol values are 0
#normalise
normalised_X = zscore(normalised_X, axis = 0, ddof = 1)

attributeNames_norm = np.copy(attributeNames)
attributeNames_norm[6] = 'log-alc'
attributeNames_norm = ['normalized ' + attribute for attribute in attributeNames_norm]

#Or without the last binary data

Y = np.copy(normalised_X[:,:-1])
N_y, M_y = Y.shape

attributeNames_y = np.copy(attributeNames_norm[:-1])

In [3]:
import numpy as np
from sklearn import model_selection
from sklearn.dummy import DummyClassifier
import torch
from sklearn.linear_model import LogisticRegression
from dtuimldmtools import train_neural_net

In [20]:
#We need to consider the possible lambdas and the number of hidden units that we want to consider in the inner loop. 
#For the baseline model there is no controlling parameter

#First try
n_hidden_units = [(2,2),(2,3)]
# print(n_hidden_units)

K_out = 2
K_in = 2
CV_out = model_selection.KFold(K_out,shuffle=True)
CV_in = model_selection.KFold(K_in,shuffle=True)

# For statistical evaluation : store outer fold predictions for the three models
yhat = []
y_true = []

# For debugging : store inner fold predictions (for each outer fold and inner fold)
ANN_full = {}

#The Error for the best model of each type of model in each outer loop
# Train_error = np.zeros((K_out,3))
Test_error= np.zeros((K_out,2))
best_h_index = np.int32(np.zeros(K_out))

for k, (train_index,test_index) in enumerate(CV_out.split(normalised_X,y)):
    print(f"#================OUTER LOOP {k+1}================#")
    #to store the new predictions of the selected model at each outer fold (to then be concatenated in yhat)
    dy = []

    #the training tests for each fold of the outer loop
    X_train = normalised_X[train_index, :]
    y_train = y[train_index]
    X_test = normalised_X[test_index, :]
    y_test = y[test_index]

    #Baseline model
    baseline = DummyClassifier(strategy='most_frequent')
    baseline.fit(X_train,y_train)
    Test_error[k,1] = 1-baseline.score(X_test,y_test)

    #INITIALIZE ERROR HANDLING
    #Error for each model in each of the loops, overwritten at each outer loop 
    ANN_Inner_test_error= np.zeros((K_in, len(n_hidden_units)))

    #average error of each model on each outter fold
    ANN_Model_out_test_error = np.zeros((K_out, len(n_hidden_units)))

    #we also need to store the sizes of the folds
    inner_fold_validate_sizes = np.zeros(K_in)

    #Inner Loop
    for i, (Inner_train_index, Inner_test_index) in enumerate(CV_in.split(X_train,y_train)):
        print(f"#================INNER LOOP {i+1}================#")
        #initialize the training and validation sets
        X_subtrain = X_train[Inner_train_index]
        y_subtrain = y[Inner_train_index]
        X_validate = X_train[Inner_test_index]
        y_validate = y_train[Inner_test_index]

        #store the size of the validation set
        inner_fold_validate_sizes[i] = X_validate.shape[0]

        #==========#
        #ANN
        #==========#

        #We convert the training and test sets to torch tensors

        X_subtrain = torch.tensor(X_subtrain, dtype=torch.float32)
        y_subtrain = torch.tensor(y_subtrain, dtype=torch.float32).reshape(-1, 1)
        X_validate = torch.tensor(X_validate, dtype=torch.float32)
        y_validate = torch.tensor(y_validate, dtype=torch.float32).reshape(-1, 1)

        #ANN cross validation loop
        for j, (n1,n2) in enumerate(n_hidden_units):
            print(f"#================ANN Model {j+1}================#")
            # The lambda-syntax defines an anonymous function, which is used here to
            # make it easy to make new networks within each cross validation fold
            model = lambda: torch.nn.Sequential(
                torch.nn.Linear(M, n1),  # M features to H hiden units
                # 1st transfer function, either Tanh or ReLU:
                torch.nn.Tanh(),  
                # torch.nn.ReLU(),
                torch.nn.Linear(n1, n2),    # 1st hidden layer to 2nd hidden layer (n2 units)
                torch.nn.ReLU(),            # 2nd activation function
                torch.nn.Linear(n2, 1),     # Output layer: n2 to 1 output neuron
                torch.nn.Sigmoid(),  # final tranfer function
            )
            #Loss function (Binary Cross Entropy)
            loss_fn = torch.nn.BCELoss()
            # Train for a maximum of 10000 steps, or until convergence
            max_iter = 10000
        
            net, final_loss, learning_curve = train_neural_net(
            model, loss_fn, X=X_subtrain, y=y_subtrain, n_replicates=1, max_iter=max_iter
            )
            y_validate_est = net(X_validate).detach().numpy()
            y_validate_pred = (y_validate_est > 0.5).astype(int)
            y_validate_np = y_validate.numpy()
            ANN_Inner_test_error[i, j] = np.sum(y_validate_pred != y_validate) / len(y_validate)
            ANN_full[(i,j,k)] = { 'predictions' : y_validate_pred.squeeze(), 'ground_truth' : y_validate_np.squeeze()}

    #Average Model Error calculation for ANN   
    ANN_Model_out_test_error = np.sum(inner_fold_validate_sizes[:,None]*ANN_Inner_test_error, axis = 0)/X_train.shape[0]
    best_h_index[k] = int(np.argmin(ANN_Model_out_test_error))

    #Retrain the best ANN model
    X_train = torch.tensor(X_train, dtype=torch.float32)
    y_train = torch.tensor(y_train, dtype=torch.float32).reshape(-1, 1)
    X_test = torch.tensor(X_test, dtype=torch.float32)
    y_test = torch.tensor(y_test, dtype=torch.float32).reshape(-1, 1)
    # best_h_index = int(best_h_index)
    n1 = n_hidden_units[best_h_index[k]][0]
    n2 = n_hidden_units[best_h_index[k]][1]
    model = lambda: torch.nn.Sequential(
                torch.nn.Linear(M, n1),  # M features to H hiden units
                # 1st transfer function, either Tanh or ReLU:
                torch.nn.ReLU(),  
                # torch.nn.ReLU(),
                torch.nn.Linear(n1, n2),    # 1st hidden layer to 2nd hidden layer (n2 units)
                torch.nn.ReLU(),            # 2nd activation function
                torch.nn.Linear(n2, 1),     # Output layer: n2 to 1 output neuron
                torch.nn.Sigmoid(),  # final tranfer function
            )
    #Loss function (Binary Cross Entropy)
    loss_fn = torch.nn.BCELoss()
    # Train for a maximum of 10000 steps, or until convergence
    max_iter = 10000
        
    net, final_loss, learning_curve = train_neural_net(
        model, loss_fn, X=X_train, y=y_train, n_replicates=1, max_iter=max_iter
    )
    y_test_est = net(X_test).detach().numpy()
    y_test_pred = (y_test_est > 0.5).astype(int)
    y_test = y_test.numpy()
    Test_error[k, 0] = np.sum(y_test_pred != y_test) / len(y_test)


    #to store the predictions
    dy.append(y_test_pred.squeeze())
    print(dy)
    dy = np.stack(dy, axis=1)
    yhat.append(dy)
    y_true.append(y_test)

yhat = np.concatenate(yhat)
y_true = np.concatenate(y_true)

#================OUTER LOOP 1================#
#================INNER LOOP 1================#
#================ANN Model 1================#

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	0.5080905	0.00016760944
		2000	0.4657387	7.742113e-05
		Final loss:
		2836	0.43919742	2.7142522e-07
#================ANN Model 2================#

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	0.46111357	0.00025716602
		2000	0.39118046	0.00013224073
		Final loss:
		2099	0.38791028	9.987634e-07
#================INNER LOOP 2================#
#================ANN Model 1================#

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	0.50056654	0.00014405923
		2000	0.4410043	0.00013946216
		Final loss:
		2740	0.4115165	5.0694484e-07
#================ANN Model 2================#

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	0.48805848	0.00018364382
		2000	0.4455217	4.46157e-05
		Final loss:
		2604	0.4349876	1.3702606e-07

	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	0.5286668	0.0005318743
		2000	0.4

Here I make some python checks for myself

In [19]:
# The lambda show that there should be no control parameter. 
# This means that the problem is inherently complicated and that there is not risk of overfitting with logistic regression.
# Maybe we can try with the logit link instead of the sigmoid function ?
print(f'optimal hidden unit parameter index : {best_h_index}')
# print(f'Optimal number of hidden units : {n_hidden_units[best_h_index]}')
print(Test_error)

optimal hidden unit parameter index : [0 0]
[[0.28138528 0.35497835]
 [0.3030303  0.33766234]]


We need to test the error handling structures.

Not much worse or much better